In [1]:
import time
import hashlib
from bank_manager import BankManager

def generate_merchant_id(name, password):
    timestamp = str(time.time())
    raw_string = name + timestamp + password
    result = hashlib.sha256(raw_string.encode()) 
    merchant_id = result.hexdigest()
    
    return merchant_id[:16]

def generate_mmid(uid, mobile_no):
    combined = uid + mobile_no
    result = hashlib.sha256(combined.encode())
    mmid = result.hexdigest()
    return mmid[:16]

In [ ]:
import socket
import random
import threading
import json
from aes_encryption import encrypt_message, decrypt_message

# Load config values
with open("config.json") as f:
    config = json.load(f)

BANK_IP = config["BANK_SERVER_IP"]
BANK_PORT = config["BANK_SERVER_PORT"]

# Lists to store merchant and user details
merchant_db = []
user_db = []

# Create server socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # Allow port reuse
server_socket.bind((BANK_IP, BANK_PORT))  # Bind to localhost and port
server_socket.listen(5)

print("🏦 Bank server is listening for connections...")

def handle_client(client_socket):
    """Handles a new client connection (User or Merchant)"""
    try:
        data = client_socket.recv(1024).decode("utf-8")  # Receive data
        details = json.loads(data)  # Convert JSON string to Python dictionary

        if details.get("type") == "merchant":
            merchant_info = json.loads(data)
            # print(f"Received Merchant Details: {merchant_info}")

            mid = generate_merchant_id(merchant_info["name"], merchant_info["password"])
            merchant_info["MID"] = mid
            merchant_db.append(merchant_info)  # Store merchant details

            # print(f"Merchant Registered: {merchant_info}")

            response = {"message": "Registration Successful", "MID": mid}
            client_socket.send(json.dumps(response).encode())

        elif details.get("type") == "user":
            user_info = json.loads(data)
            # print(f"Received User Details: {user_info}")

            uid = generate_merchant_id(user_info["name"], user_info["password"])
            user_info["UID"] = uid
            user_db.append(user_info) 

            # print(f"User Registered: {user_info}")

            response = {"message": "Registration Successful", "UID": uid}
            client_socket.send(json.dumps(response).encode())
            
        elif details.get("type") == "check_merchant":
            mid = details.get("merchant_id")
            exists = any(merchant["MID"] == mid for merchant in merchant_db)
            if exists:
                client_socket.send("True".encode())
            else:
                client_socket.send("False".encode())
                
        elif details.get("type") == "check_user":
            uid = details.get("user_id")
            exists = any(user["UID"] == uid for user in user_db)
            if exists:
                client_socket.send("True".encode())
            else:
                client_socket.send("False".encode())    
        
        elif details.get("type") == "MMID_Generation":
            uid = details.get("user_id")
            mobile_num = details.get("mobile_number")
            mmid = generate_mmid(uid, mobile_num)
            for user in user_db:
                if user["UID"] == uid:
                    user["MMID"] = mmid
                    # print(f"User Registered: {user}")
            response = {"message": "Registration Successful", "MMID": mmid}
            client_socket.send(json.dumps(response).encode())
            
        elif details["type"] == "transaction":
            encrypt_mmid = details["mmid"]
            encrypt_pin = details["pin"]
            encrypt_amount = details["amount"]
            mid = details["merchant_id"]
            
            mmid = decrypt_message(encrypt_mmid)
            pin = decrypt_message(encrypt_pin)
            amount = decrypt_message(encrypt_amount)
            
            new_merch = {}
            for merchant in merchant_db:
                if merchant["MID"] == mid:
                    new_merch = merchant
                    break
            
            user_found = False
            for user in user_db:
                if user["MMID"] == mmid and user["pin"] == pin:
                    user_found = True
                    balance = float(user["balance"])
                    merch_balance = float(new_merch["balance"])
                    amount = float(amount)
                    
                    if balance >= amount:
                        user["balance"] = balance - amount
                        new_merch["balance"] = merch_balance + amount       
                        response = {
                            "status": "Transaction has been successful!",
                        }
                    else:
                        response = {
                            "status": "Transaction failed due to insufficient balance!"
                        }
                    break
            if not user_found:
                response = {
                    "status": "Transaction failed! User Not Found",
                    "message": "Invalid MMID or PIN"
                }

            client_socket.send(json.dumps(response).encode())
            
        else:
            client_socket.send("Error: Invalid request".encode("utf-8"))
        
        print(user_db)
        print(merchant_db)
        
    except Exception as e:
        print(f"❌ Error: {e}")

    finally:
        client_socket.close()  # Close connection

print(user_db)
print(merchant_db)

try:
    while True:
        client_socket, addr = server_socket.accept()
        print(f"🔗 New connection from {addr}")
        threading.Thread(target=handle_client, args=(client_socket,)).start()

except KeyboardInterrupt:
    print("\nServer shutting down gracefully...")
    server_socket.close()

🏦 Bank server is listening for connections...
[]
[]
🔗 New connection from ('127.0.0.1', 64130)
[]
[{'type': 'merchant', 'name': 'meghna', 'password': 'abhdgjv', 'balance': 2000.0, 'ifsc': 'HDFC001', 'MID': 'e48944f2fa16061f'}]
🔗 New connection from ('127.0.0.1', 64135)
[{'type': 'user', 'name': 'lucky', 'password': 'fc65vuyi', 'balance': 2000.0, 'ifsc': 'ICICI001', 'pin': '1234', 'UID': '6422a62f4bce10c4'}]
[{'type': 'merchant', 'name': 'meghna', 'password': 'abhdgjv', 'balance': 2000.0, 'ifsc': 'HDFC001', 'MID': 'e48944f2fa16061f'}]
🔗 New connection from ('127.0.0.1', 64138)
[{'type': 'user', 'name': 'lucky', 'password': 'fc65vuyi', 'balance': 2000.0, 'ifsc': 'ICICI001', 'pin': '1234', 'UID': '6422a62f4bce10c4'}]
[{'type': 'merchant', 'name': 'meghna', 'password': 'abhdgjv', 'balance': 2000.0, 'ifsc': 'HDFC001', 'MID': 'e48944f2fa16061f'}]
🔗 New connection from ('127.0.0.1', 64139)
[{'type': 'user', 'name': 'lucky', 'password': 'fc65vuyi', 'balance': 2000.0, 'ifsc': 'ICICI001', 'pin':